# Deep Learning
## 2️⃣ TensorFlow and Neural Network

### How deep learning model learns

Deep Learning model is trained to find **weights** that minimalizes the value of **Loss function** using various optimization methods.

#### Forward propagation
**Forward propagation** means calculating and storing variables in order from the input layer to the output layer of the neural network model. Deep learning model caculates predicted value by forward propagation.

#### Gradient Descent
Deep Learning uses **Gradient Descent** to minimalize the value of Loss function. The gradient value of each weight may be obtained through **backpropagation**.

#### The learning order of the deep learning model
1. Calculate predicted value using forward propagation.
2. Calculate Loss function.
3. Update weight using backpropagation to minimalize the value of loss fuction.
4. Repeate 1~3 to get the value of weights that minimalize the value of loss function.

### Using TensorFlow to preprocess data

Tensorflow's deep learning model gets data in the form of **Tensor**, which is multi-dimensional array object of tensorflow.   
You can create tensor using ```tf.data.Dataset``` function.

In [ ]:
# Use pandas to get data
df = pd.read_csv("data.csv")
feature = df.drop(columns=['label'])
label = df['label']

# Convert data to tensor
dataset = tf.data.Dataset.from_tensor_slices((feature.values, label.values))

Additional preprocessing operations called **Epoch** and **Batch** are required to generate data used for deep learning.

- **Epoch**: One Epoch is when an **entire** dataset is passed forward and backward through the neural network only **once**.
- **Batch size**: Total number of training examples present in a single **batch**.
- **Iteration**: The number of passes to complete one epoch.

For example, if the number of total data is 1000 and the batch size is 100, the model will learn 100 data per iteration, and 1 epoch will mean 10 iterations.


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((feature.values, label.values))

# Set batch size to 32
dataset = dataset.batch(32)

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

np.random.seed(100)
tf.random.set_seed(100)

# Import data in the form of DataFrame.
df = pd.read_csv("./Advertising.csv")

print('Samples of original data :')
print(df.head(),'\n')

# Delete meaningless column.
df = df.drop(columns=['Unnamed: 0'])

"""
1. Save 'Sales' column as label data in Y, and the rest as feature data in X.
"""
X = df.drop(columns=['Sales'])
Y = df['Sales']

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3)

"""
2. Convert data for training into tf.data.Dataset form. Use from_tensor_slices function.
"""
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y.values))

"""
3. Set batch size as 5, and shuffe data.
"""
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)

# Take 1 batch and assign feature data and label data of it to train_features_batch and label_batch respectively.
[(train_features_batch, label_batch)] = train_ds.take(1)

# Print each varaible.
print('\nFB, TV, Newspaper batch data:\n',train_features_batch)
print('Sales batch data:',label_batch)


Samples of original data :
   Unnamed: 0     TV  Radio  Newspaper  Sales
0           1  230.1   37.8       69.2   22.1
1           2   44.5   39.3       45.1   10.4
2           3   17.2   45.9       69.3    9.3
3           4  151.5   41.3       58.5   18.5
4           5  180.8   10.8       58.4   12.9 


FB, TV, Newspaper batch data:
 tf.Tensor(
[[296.4  36.3 100.9]
 [228.   37.7  32. ]
 [  5.4  29.9   9.4]
 [ 57.5  32.8  23.5]
 [240.1   7.3   8.7]], shape=(5, 3), dtype=float64)
Sales batch data: tf.Tensor([23.8 21.5  5.3 11.8 13.2], shape=(5,), dtype=float64)


### Implementing deep learning using TensorFlow & Keras

1. Creating model class object

Use ```tf.keras.models.Sequential()```.

2. Composing each layer of the model

Use ```tf.keras.layers.Dense(units, activation)```.    

**units**: the number of nodes in a layer   
**activation**: activation function to use (i.e. sigmoid, tanh, ...)

Input layers need information about type of input data. So, set ```input_shape``` or ```input_dim``` to proper value.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_dim = 2, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

You can add more layers to the model by using ```[model].add(tf.keras.layers.Dense(units, activation))```.

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(10, input_dim=2, activation='sigmoid'))
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

3. Training the model

Use ```[model].compile(optimizer, loss)``` to set a model learning method,  and use ```[model].fit(x, y)``` to train the model.

For regression model, **MSE** is generally used as loss function, and **Cross Entropy** is generally used as loss function for classification model. 

In [ ]:
# Set MSE as loss function, and use SGD as optimizer.
model.compile(loss='mean_square_error', optimizer='SGD')

# Input data in dataset, set epochs as 100, then train the model.
model.fit(dataset, epochs=100)

4. Testing the model

Use ```[model].evaluate(x, y)``` to test the model, and use ```[model].predict(x)``` to predict values using the model.

In [ ]:
model.evaluate(X_test, Y_test)
predicted_labels_test = model.predict(X_test)

#### Example Code

In [3]:
"""
1. Create deep learning model using functions of Keras.
"""
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=(3,)),
    tf.keras.layers.Dense(1)
    ])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                40        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
"""
2. Train the model
    
step1. Set loss function as 'mean_squared_error', and optimizer as 'adam'
       
step2. Train the model using fit method. 
'verbose' parameter sets what is output on the screen during learning.
(0: No output 1: progress bar 2: one line per one epoch)

fit 메서드를 사용하여 Dataset으로 변환된 학습용 데이터를 학습합니다.
       epochs는 100으로 설정합니다.
"""
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(train_ds, epochs=100, verbose=2)

Epoch 1/100
28/28 - 0s - loss: 41.7773
Epoch 2/100
28/28 - 0s - loss: 21.6253
Epoch 3/100
28/28 - 0s - loss: 16.2435
Epoch 4/100
28/28 - 0s - loss: 12.3379
Epoch 5/100
28/28 - 0s - loss: 10.2867
Epoch 6/100
28/28 - 0s - loss: 8.0436
Epoch 7/100
28/28 - 0s - loss: 6.7272
Epoch 8/100
28/28 - 0s - loss: 6.1394
Epoch 9/100
28/28 - 0s - loss: 5.5167
Epoch 10/100
28/28 - 0s - loss: 4.8297
Epoch 11/100
28/28 - 0s - loss: 4.5993
Epoch 12/100
28/28 - 0s - loss: 4.4995
Epoch 13/100
28/28 - 0s - loss: 4.4540
Epoch 14/100
28/28 - 0s - loss: 4.3003
Epoch 15/100
28/28 - 0s - loss: 4.2078
Epoch 16/100
28/28 - 0s - loss: 4.1856
Epoch 17/100
28/28 - 0s - loss: 4.3536
Epoch 18/100
28/28 - 0s - loss: 4.4149
Epoch 19/100
28/28 - 0s - loss: 4.3582
Epoch 20/100
28/28 - 0s - loss: 4.1821
Epoch 21/100
28/28 - 0s - loss: 4.1871
Epoch 22/100
28/28 - 0s - loss: 4.2035
Epoch 23/100
28/28 - 0s - loss: 4.0640
Epoch 24/100
28/28 - 0s - loss: 4.0174
Epoch 25/100
28/28 - 0s - loss: 4.1636
Epoch 26/100
28/28 - 0s - los

In [6]:
"""
3. Calculate loss value for test data using evaluate method.
"""
loss = model.evaluate(test_X, test_Y, verbose=1)

"""
4. Predict the value for test data using predict method.
"""
predictions = model.predict(test_X)

# Print the result.
print("Loss value of test data: ", loss)
for i in range(5):
    print("Actual value of the test data #%d: %f" % (i, test_Y.iloc[i]))
    print("Predicted value of the test data #%d: %f" % (i, predictions[i][0]))

2/2 [==============================] - 0s 2ms/step - loss: 1.9413
Loss value of test data:  1.9413315057754517
Actual value of the test data #0: 6.600000
Predicted value of the test data #0: 10.639652
Actual value of the test data #1: 20.700000
Predicted value of the test data #1: 20.478485
Actual value of the test data #2: 17.200000
Predicted value of the test data #2: 17.351406
Actual value of the test data #3: 19.400000
Predicted value of the test data #3: 19.829838
Actual value of the test data #4: 21.800000
Predicted value of the test data #4: 21.564997


In [ ]:
# Iris data로 실습한 내용도 정리할 것!